In [1]:
from openai import OpenAI

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [7]:
import os

In [8]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [13]:
response = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages = [
        {
            "role": "system", "content": f"""
            You are an expert in categorising IT support tickets. Given the support ticket, you need categorise each ticket into one of the 
            categories listed below:
            1. Hardware
            2. Software
            3. Others
    
            Response in one of the above words only. 
            """},
        {
            "role": "user", "content": f"My monitor won't turn on"
        }
    ], temperature = 0
)

print(response.choices[0].message.content)

Hardware


In [16]:
eval_test = client.evals.create(
    name = "IT Ticket Categorisation",
    data_source_config = {
        "type": "custom",
        "item_schema": {
            "type": "object",
            "properties": {
                "ticket_text": {"type": "string"},
                "correct_label": {"type": "string"}
            },
            "required": ["ticket_text", "correct_label"]
        },
        "include_sample_schema": True
    },

    testing_criteria = [
        {
            "type": "string_check",
            "name": "Match to human label",
            "input": "{{sample.output_text}}",
            "operation": "eq",
            "reference": "{{item.correct_label}}"
        }
    ]
)

print(eval_test)

EvalCreateResponse(id='eval_6940b036ee948191a812f11f980d4935', created_at=1765847094, data_source_config=EvalCustomDataSourceConfig(schema_={'type': 'object', 'properties': {'item': {'type': 'object', 'properties': {'ticket_text': {'type': 'string'}, 'correct_label': {'type': 'string'}}, 'required': ['ticket_text', 'correct_label']}, 'sample': {'type': 'object', 'properties': {'model': {'type': 'string'}, 'choices': {'type': 'array', 'items': {'type': 'object', 'properties': {'message': {'type': 'object', 'properties': {'role': {'type': 'string', 'enum': ['assistant']}, 'content': {'type': ['string', 'array', 'null']}, 'refusal': {'type': ['boolean', 'null']}, 'tool_calls': {'type': ['array', 'null'], 'items': {'type': 'object', 'properties': {'type': {'type': 'string', 'enum': ['function']}, 'function': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'arguments': {'type': 'string'}}, 'required': ['name', 'arguments']}, 'id': {'type': 'string'}}, 'required': ['type', 'fun

In [15]:
file_upload = client.files.create(
    file = open("/Users/Rakshit.Lodha/evals/tickets.jsonl","rb"),
    purpose = "evals"
)

print(file_upload)

FileObject(id='file-BW89c77fZFoMrq8dG4Xb3k', bytes=268, created_at=1765846820, filename='tickets.jsonl', object='file', purpose='evals', status='processed', expires_at=None, status_details=None)


In [17]:
eval_run = client.evals.runs.create(
    "eval_6940b036ee948191a812f11f980d4935",
    name = "Categorisation",
    data_source = {
        "type": "responses",
        "model": "gpt-3.5-turbo",
        "input_messages": {
            "type": "template",
            "template": [
                {"role": "developer", "content": "You are an expert in categorizing IT support tickets. Given the support ticket below, categorize the request into one of 'Hardware', 'Software', or 'Other'. Respond with only one of those words."},
                {"role": "user", "content": "{{ item.ticket_text }}"},
            ]
        },
        "source": {"type": "file_id", "id": "file-BW89c77fZFoMrq8dG4Xb3k"}
    }
)

print(eval_run)

RunCreateResponse(id='evalrun_6940b117c0f481919adf813d0d40f93f', created_at=1765847319, data_source=DataSourceResponses(source=DataSourceResponsesSourceFileID(id='file-BW89c77fZFoMrq8dG4Xb3k', type='file_id'), type='responses', input_messages=DataSourceResponsesInputMessagesTemplate(template=[DataSourceResponsesInputMessagesTemplateTemplateEvalItem(content="You are an expert in categorizing IT support tickets. Given the support ticket below, categorize the request into one of 'Hardware', 'Software', or 'Other'. Respond with only one of those words.", role='developer', type='message'), DataSourceResponsesInputMessagesTemplateTemplateEvalItem(content='{{ item.ticket_text }}', role='user', type='message')], type='template'), model='gpt-3.5-turbo', sampling_params=None), error=None, eval_id='eval_6940b036ee948191a812f11f980d4935', metadata={}, model='gpt-3.5-turbo', name='Categorisation', object='eval.run', per_model_usage=None, per_testing_criteria_results=None, report_url='https://platfo

In [32]:
response_1 = client.chat.completions.create(
    model = "gpt-4o",
    messages = [
        {"role": "system", "content": f""" 
        You are a helpful advisor, who is supposed to answer life's questions thoughtfully and empathetically.
        But at the same time ensure that you're not being too generic and giving actionable advice to the user

        Make sure the answer is conscise and does not hold back 
        """},
        {"role": "user", "content": "Is it wrong to delay marriage because I want to focus on my career right now?"}
    ], temperature = 0.7
)

print(response_1.choices[0].message.content)

It's not wrong at all to delay marriage to focus on your career. Prioritizing your personal and professional goals is a valid choice and can lead to greater fulfillment in the long run. It's vital to communicate openly with your partner about your intentions and ensure you're both aligned in your life goals. Consider setting a realistic timeline for when you might revisit the idea of marriage, and be mindful of balancing your career ambitions with nurturing your relationship to ensure both areas receive the attention they deserve.


In [30]:
eval_test_1 = client.evals.create(
    name = "Helpful assistant",
    data_source_config = {
        "type": "custom",
        "item_schema": {
            "type": "object",
            "properties": {
                "question": {"type": "string"},
                "ideal_answer": {"type": "string"}
            },
            "required": ["question","ideal_answer"]
        },
        "include_sample_schema": True
    },
    testing_criteria = [
        {
            "type": "score_model",
            "name": "Model Score",
            "input": [
                {
                    "role": "system",
                    "content": """You are an expert grader and you're supposed to grade whether the {{sample.output_text}}
                    is following the reference provided in the form of {{item.ideal_answer}}. You are supposed to follow the following 
                    parameters, which are as follows:
                    1. Empathy
                    2. Concise
                    3. Actionable

                    Finally rate it out 5 based on the above paramters. 
                    - If it is following all 3, rate it 5
                    - If it is following 2, rate it 3
                    - If it is following <2, rate it 0

                    Output only a single integer from {0, 3, 5}. Do not explain your reasoning.

                    If the response gives absolute advice (‘you must’, ‘definitely do X’), score = 0 regardless
                    """
                },
                {
                    "role": "user",
                    "content": "Reference: {{item.ideal_answer}}. Model Output: {{sample.output_text}}"
                }
            ],
            "pass_threshold": 3,
            "model": "gpt-4o",
            "range": [0,5]
        }
    ]
)

print(eval_test_1)

EvalCreateResponse(id='eval_6941a794dab88191b3cac85732805a04', created_at=1765910420, data_source_config=EvalCustomDataSourceConfig(schema_={'type': 'object', 'properties': {'item': {'type': 'object', 'properties': {'question': {'type': 'string'}, 'ideal_answer': {'type': 'string'}}, 'required': ['question', 'ideal_answer']}, 'sample': {'type': 'object', 'properties': {'model': {'type': 'string'}, 'choices': {'type': 'array', 'items': {'type': 'object', 'properties': {'message': {'type': 'object', 'properties': {'role': {'type': 'string', 'enum': ['assistant']}, 'content': {'type': ['string', 'array', 'null']}, 'refusal': {'type': ['boolean', 'null']}, 'tool_calls': {'type': ['array', 'null'], 'items': {'type': 'object', 'properties': {'type': {'type': 'string', 'enum': ['function']}, 'function': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'arguments': {'type': 'string'}}, 'required': ['name', 'arguments']}, 'id': {'type': 'string'}}, 'required': ['type', 'function', 

In [27]:
file_upload_1 = client.files.create(
    file = open("/Users/Rakshit.Lodha/evals/response.jsonl","rb"),
    purpose = "evals"
)

print(file_upload_1)

FileObject(id='file-QPbtrPmabv46T6BEv9RAXC', bytes=2676, created_at=1765910121, filename='response.jsonl', object='file', purpose='evals', status='processed', expires_at=None, status_details=None)


In [31]:
eval_run_1 = client.evals.runs.create(
    "eval_6941a794dab88191b3cac85732805a04",
    name = "Model Response Check V1",
    data_source = {
        "type": "responses",
        "model": "gpt-4o",
        "input_messages": {
            "type": "template",
            "template": [
                {"role": "developer", "content": """ You are a helpful advisor, 
                who is supposed to answer life's questions thoughtfully and empathetically.
                But at the same time ensure that you're not being too generic and giving actionable advice to the user
                Make sure the answer is conscise and does not hold back """},
                {"role": "user", "content": "{{item.question}}"}
            ]
        },
        "source": {"type": "file_id", "id": "file-QPbtrPmabv46T6BEv9RAXC"}
    }
)


print(eval_run_1)

RunCreateResponse(id='evalrun_6941a7a18914819187d63ed7a0877ea5', created_at=1765910433, data_source=DataSourceResponses(source=DataSourceResponsesSourceFileID(id='file-QPbtrPmabv46T6BEv9RAXC', type='file_id'), type='responses', input_messages=DataSourceResponsesInputMessagesTemplate(template=[DataSourceResponsesInputMessagesTemplateTemplateEvalItem(content=" You are a helpful advisor, \n                who is supposed to answer life's questions thoughtfully and empathetically.\n                But at the same time ensure that you're not being too generic and giving actionable advice to the user\n                Make sure the answer is conscise and does not hold back ", role='developer', type='message'), DataSourceResponsesInputMessagesTemplateTemplateEvalItem(content='{{item.question}}', role='user', type='message')], type='template'), model='gpt-4o', sampling_params=None), error=None, eval_id='eval_6941a794dab88191b3cac85732805a04', metadata={}, model='gpt-4o', name='Model Response Ch